In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
 from tensorflow.keras.applications import VGG16
import numpy as np
import os
vgg_base = VGG16(weights='imagenet',    # use weights for ImageNet
                 include_top=False,     # drop the Dense layers!
                 input_shape=(150, 150, 3))
print(vgg_base.summary())

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______________________________________________________________

In [ ]:
import tensorflow as tf

from tensorflow import keras

tf.keras.Sequential(layers=None, name=None)

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam



model = tf.keras.Sequential([

        vgg_base,

        Flatten(),
        Dropout(0.50),
        Dense(1024, activation='relu'),
        Dropout(0.20),
        Dense(512, activation='relu'),
        Dropout(0.10),
        Dense(1, activation='sigmoid')
    ])

In [ ]:
vgg_base.trainable = False


In [ ]:
model.compile(optimizer=Adam(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
def build_model():
    from tensorflow.keras.layers import Flatten, Dense, Dropout
    from tensorflow.keras.optimizers import Adam

    model = tf.keras.models.Sequential([
        # our vgg16_base model added as a layer
        vgg_base,
        # here is our custom prediction layer (same as before)
        Flatten(),
        Dropout(0.50),
        Dense(1024, activation='relu'),
        Dropout(0.20),
        Dense(512, activation='relu'),
        Dropout(0.10),
        Dense(1, activation='sigmoid')
    ])

    # mark the top 2 layers of vgg_base as trainable
    trainable = False
    for layer in vgg_base.layers:
       if layer.name == 'block5_conv2':
           trainable = True
    layer.trainable = trainable

    model.compile(optimizer=Adam(lr=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
images_root = "/content/gdrive/My Drive/class"
train_root = os.path.join(images_root,'train')
eval_root = os.path.join(images_root,'validate')
test_root=os.path.join(images_root,'test')
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [ ]:
model = build_model()
# Step-1: Create data generators
# -----------------------------------
# build our image data generators --
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
# NOTE: no image aug for eval & test datagenerators
eval_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# ALL IMAGES TO BE RESIZED to (150,150,3)
train_generator = train_datagen.flow_from_directory(
    train_root,
    target_size=(150,150),
    batch_size=10,
    class_mode='binary')
eval_generator = eval_datagen.flow_from_directory(
    eval_root,
    target_size=(150,150),
    batch_size=10,
    class_mode='binary')



test_generator = test_datagen.flow_from_directory(
    test_root,
    target_size=(150,150),
    batch_size=10,
    class_mode='binary')
val_steps = eval_generator.n // 10
# train model on generator with batch size = 4
train_steps = train_generator.n // 10


test_steps = test_generator.n // 10
history=model.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=50,
    validation_data=eval_generator,
    validation_steps=val_steps)

Found 342 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Found 68 images belonging to 2 classes.
Epoch 1/50
34/34 [==============================] - 141s 4s/step - loss: 0.7434 - accuracy: 0.5964 - val_loss: 0.6035 - val_accuracy: 0.8000
Epoch 2/50
34/34 [==============================] - 94s 3s/step - loss: 0.4758 - accuracy: 0.7831 - val_loss: 0.5376 - val_accuracy: 0.8000
Epoch 3/50
34/34 [==============================] - 97s 3s/step - loss: 0.4523 - accuracy: 0.7711 - val_loss: 0.6479 - val_accuracy: 0.8000
Epoch 4/50
34/34 [==============================] - 94s 3s/step - loss: 0.3472 - accuracy: 0.8735 - val_loss: 0.7180 - val_accuracy: 0.8000
Epoch 5/50
34/34 [==============================] - 94s 3s/step - loss: 0.3150 - accuracy: 0.8554 - val_loss: 0.5933 - val_accuracy: 0.8000
Epoch 6/50
34/34 [==============================] - 94s 3s/step - loss: 0.3110 - accuracy: 0.8584 - val_loss: 0.6618 - val_accuracy: 0.8000
Epoch 7/50
34/34 [====================

In [ ]:
# Step-4: evaluate model's performance on train/eval/test datasets
# ----------------------------------------------------------------
loss, acc = model.evaluate_generator(train_generator,
                     steps=train_steps, verbose=1)
print('Training data  -> loss: %.3f, acc: %.3f' % (loss, acc))
loss, acc = model.evaluate_generator(eval_generator,
                     steps=val_steps, verbose=1)
print('Cross-val data -> loss: %.3f, acc: %.3f' % (loss, acc))
loss, acc = model.evaluate_generator(test_generator,
                      steps=test_steps, verbose=1)
print('Testing data   -> loss: %.3f, acc: %.3f' % (loss, acc))

Instructions for updating:
Please use Model.evaluate, which supports generators.
34/34 [==============================] - 91s 3s/step - loss: 0.1643 - accuracy: 0.9382
Training data  -> loss: 0.164, acc: 0.938
1/1 [==============================] - 0s 1ms/step - loss: 1.1994 - accuracy: 0.6000
Cross-val data -> loss: 1.199, acc: 0.600
6/6 [==============================] - 29s 5s/step - loss: 0.2503 - accuracy: 0.9000
Testing data   -> loss: 0.250, acc: 0.900
